**Workshop**

Screening with technical analysis

## Import library

In [249]:
import yfinance as yf
import mplfinance as mpf
import pandas as pd
import numpy as np
import pandas_ta as ta

## Import data

In [250]:
# define asset and date
stocks = 'PTT.BK EA.BK COM7.BK KBANK.BK RCL.BK'

from datetime import date

today = date.today().strftime("%Y-%m-%d")


start='2019-01-01' 
end=today

In [251]:
raw_df = yf.download(tickers=stocks, start=start, end=end)

[*********************100%***********************]  5 of 5 completed


In [252]:
raw_df

Adj Close                                                Close  \
              COM7.BK      EA.BK    KBANK.BK     PTT.BK     RCL.BK COM7.BK   
Date                                                                         
2019-01-02   6.764174  41.742096  165.534546  38.468914   3.681555    7.45   
2019-01-03   5.947025  42.474419  166.429337  37.845093   3.712235    6.55   
2019-01-04   6.128614  43.206730  166.429337  38.884792   3.712235    6.75   
2019-01-07   6.083216  43.939049  168.218933  39.924488   3.742914    6.70   
2019-01-08   6.219408  44.183159  166.876724  39.924488   3.696895    6.85   
...               ...        ...         ...        ...        ...     ...   
2022-12-19  31.750000  96.000000  144.000000  31.250000  30.000000   31.75   
2022-12-20  32.250000  96.250000  143.000000  31.250000  29.500000   32.25   
2022-12-21  32.000000  95.750000  144.500000  31.000000  30.000000   32.00   
2022-12-22  32.250000  95.500000  145.000000  31.500000  30.500000   32.25   
2022-12-23  32.000000  93.750000  144.000000  31.500000  30.500000   32.00   

                                          ...    Open                         \
            EA.BK KBANK.BK PTT.BK RCL.BK  ... COM7.BK  EA.BK KBANK.BK PTT.BK   
Date                                      ...                                  
2019-01-02  42.75    185.0  46.25   4.80  ...    7.80  42.75    185.0  46.25   
2019-01-03  43.50    186.0  45.50   4.84  ...    7.50  43.25    186.5  46.50   
2019-01-04  44.25    186.0  46.75   4.84  ...    6.35  43.25    185.5  45.50   
2019-01-07  45.00    188.0  48.00   4.88  ...    6.85  44.50    186.0  47.25   
2019-01-08  45.25    186.5  48.00   4.82  ...    6.75  45.00    187.5  48.25   
...           ...      ...    ...    ...  ...     ...    ...      ...    ...   
2022-12-19  96.00    144.0  31.25  30.00  ...   32.25  96.50    144.0  31.00   
2022-12-20  96.25    143.0  31.25  29.50  ...   32.00  96.25    144.0  31.00   
2022-12-21  95.75    144.5  31.00  30.00  ...   33.00  96.50    143.5  31.25   
2022-12-22  95.50    145.0  31.50  30.50  ...   32.25  96.25    145.0  31.00   
2022-12-23  93.75    144.0  31.50  30.50  ...   31.75  95.00    144.0  31.50   

                      Volume                                        
           RCL.BK    COM7.BK     EA.BK KBANK.BK    PTT.BK   RCL.BK  
Date                                                                
2019-01-02   4.74   20118200  11894900  4410900  30481900    33500  
2019-01-03   4.80   56974000  18362000  4601300  53438400    74300  
2019-01-04   4.78  141061800  12736800  8782100  60708300    33700  
2019-01-07   4.88   44884600  15897400  7171800  86356300    32300  
2019-01-08   4.88   52647000  11499600  4042100  50671500    18500  
...           ...        ...       ...      ...       ...      ...  
2022-12-19  30.00    5442500   4760400  3687200  47133700  3597000  
2022-12-20  30.00   14485000   4236100  6398700  48572100  2773800  
2022-12-21  29.75   12638600   3221600  5608700  76194000  1835700  
2022-12-22  30.25    4587600   2256900  7344700  30432600  4904300  
2022-12-23  30.50    4945800   8811400  7343700  37979900  2340300  

[964 rows x 30 columns]

In [253]:
df = raw_df.copy()

## Added TA on multi assets

In [254]:
technicals = ['sma_10', 'sma_25', 'vwma', '52week_high', '52week_low', 'macd']
tickers = stocks.split(' ')

for ticker in tickers:
  for t in technicals:
    # SMA
    if t[:3] == 'sma': # if 1st 3 letters == 'sma'
      sma_length = int(t[-2:]) # last 2 letters (length of sma)
      df[(t, ticker)] = ta.sma(df.loc[:,('Close', ticker)], length=sma_length)
    # 52week-h or low
    elif t[:6] == '52week': # if 1st 5 letters == '52week'
      if t[-4:] == 'high': # last 1 letters == 'h' else 'l'
        df[(t, ticker)] = df.loc[:, ('Close', ticker)].rolling(window=52*7, min_periods=1).max()
        df[('52week_high_count', ticker)] = np.cumsum(df[('52week_high', ticker)].diff() != 0)
      else:
        df[(t, ticker)] = df.loc[:, ('Close', ticker)].rolling(window=52*7, min_periods=1).min()
        df[('52week_low_count', ticker)] = np.cumsum(df[('52week_low', ticker)].diff() != 0)
    elif t == 'macd':
        df[(t, ticker)] = ta.sma(df.loc[:,('Close', ticker)], length=12) - ta.sma(df.loc[:,('Close', ticker)], length=26)
    # vwma
    else:
      df[(t, ticker)] = ta.vwma(df.loc[:,('Close', ticker)], df.loc[:,('Volume', ticker)])

# reference: https://stackoverflow.com/questions/70619880/pandas-ta-with-multiindex-dataframe

In [255]:
df

Adj Close                                                Close  \
              COM7.BK      EA.BK    KBANK.BK     PTT.BK     RCL.BK COM7.BK   
Date                                                                         
2019-01-02   6.764174  41.742096  165.534546  38.468914   3.681555    7.45   
2019-01-03   5.947025  42.474419  166.429337  37.845093   3.712235    6.55   
2019-01-04   6.128614  43.206730  166.429337  38.884792   3.712235    6.75   
2019-01-07   6.083216  43.939049  168.218933  39.924488   3.742914    6.70   
2019-01-08   6.219408  44.183159  166.876724  39.924488   3.696895    6.85   
...               ...        ...         ...        ...        ...     ...   
2022-12-19  31.750000  96.000000  144.000000  31.250000  30.000000   31.75   
2022-12-20  32.250000  96.250000  143.000000  31.250000  29.500000   32.25   
2022-12-21  32.000000  95.750000  144.500000  31.000000  30.000000   32.00   
2022-12-22  32.250000  95.500000  145.000000  31.500000  30.500000   32.25   
2022-12-23  32.000000  93.750000  144.000000  31.500000  30.500000   32.00   

                                          ... 52week_low_count      macd  \
            EA.BK KBANK.BK PTT.BK RCL.BK  ...         KBANK.BK  KBANK.BK   
Date                                      ...                              
2019-01-02  42.75    185.0  46.25   4.80  ...                1       NaN   
2019-01-03  43.50    186.0  45.50   4.84  ...                1       NaN   
2019-01-04  44.25    186.0  46.75   4.84  ...                1       NaN   
2019-01-07  45.00    188.0  48.00   4.88  ...                1       NaN   
2019-01-08  45.25    186.5  48.00   4.82  ...                1       NaN   
...           ...      ...    ...    ...  ...              ...       ...   
2022-12-19  96.00    144.0  31.25  30.00  ...               49 -0.727564   
2022-12-20  96.25    143.0  31.25  29.50  ...               49 -0.660256   
2022-12-21  95.75    144.5  31.00  30.00  ...               49 -0.483974   
2022-12-22  95.50    145.0  31.50  30.50  ...               49 -0.294872   
2022-12-23  93.75    144.0  31.50  30.50  ...               49 -0.108974   

            sma_10 sma_25       vwma 52week_high 52week_high_count 52week_low  \
            RCL.BK RCL.BK     RCL.BK      RCL.BK            RCL.BK     RCL.BK   
Date                                                                            
2019-01-02     NaN    NaN        NaN        4.80                 1        4.8   
2019-01-03     NaN    NaN        NaN        4.84                 2        4.8   
2019-01-04     NaN    NaN        NaN        4.84                 2        4.8   
2019-01-07     NaN    NaN        NaN        4.88                 3        4.8   
2019-01-08     NaN    NaN        NaN        4.88                 3        4.8   
...            ...    ...        ...         ...               ...        ...   
2022-12-19  29.250  30.14  29.292451       62.50                57       26.5   
2022-12-20  29.225  30.06  29.274408       62.50                57       26.5   
2022-12-21  29.325  30.00  29.333796       62.50                57       26.5   
2022-12-22  29.475  29.93  29.526505       62.50                57       26.5   
2022-12-23  29.650  29.87  29.645179       62.50                57       26.5   

           52week_low_count      macd  
                     RCL.BK    RCL.BK  
Date                                   
2019-01-02                1       NaN  
2019-01-03                1       NaN  
2019-01-04                1       NaN  
2019-01-07                1       NaN  
2019-01-08                1       NaN  
...                     ...       ...  
2022-12-19              116 -0.711538  
2022-12-20              116 -0.782051  
2022-12-21              116 -0.724359  
2022-12-22              116 -0.623397  
2022-12-23              116 -0.431090  

[964 rows x 70 columns]

## Screening Assets can get max count of 52week high

In [256]:
df['52week_high_count'].max().sort_values(ascending=False)

COM7.BK     85
RCL.BK      57
KBANK.BK    46
EA.BK       44
PTT.BK      30
dtype: int64

## Overall result of each indicators

In [ ]:
df.loc[['2022-12-23']]

Adj Close                                 Close                  \
             COM7.BK  EA.BK KBANK.BK PTT.BK RCL.BK COM7.BK  EA.BK KBANK.BK   
Date                                                                         
2022-12-23      32.0  93.75    144.0   31.5   30.5    32.0  93.75    144.0   

                          ... 52week_low_count      macd sma_10 sma_25  \
           PTT.BK RCL.BK  ...         KBANK.BK  KBANK.BK RCL.BK RCL.BK   
Date                      ...                                            
2022-12-23   31.5   30.5  ...               49 -0.108974  29.65  29.87   

                 vwma 52week_high 52week_high_count 52week_low  \
               RCL.BK      RCL.BK            RCL.BK     RCL.BK   
Date                                                             
2022-12-23  29.645179        62.5                57       26.5   

           52week_low_count     macd  
                     RCL.BK   RCL.BK  
Date                                  
2022-12-23              116 -0.43109  

[1 rows x 70 columns]

In [262]:
df.loc[['2022-12-23']].stack()

52week_high  52week_high_count  52week_low  \
Date                                                              
2022-12-23 COM7.BK          43.0                 85        27.0   
           EA.BK           100.0                 44        57.5   
           KBANK.BK        170.5                 46       102.0   
           PTT.BK           40.5                 30        31.0   
           RCL.BK           62.5                 57        26.5   

                     52week_low_count  Adj Close   Close    High     Low  \
Date                                                                       
2022-12-23 COM7.BK                 57      32.00   32.00   32.25   31.75   
           EA.BK                   31      93.75   93.75   95.25   92.50   
           KBANK.BK                49     144.00  144.00  144.50  143.00   
           PTT.BK                  31      31.50   31.50   31.75   31.25   
           RCL.BK                 116      30.50   30.50   30.75   30.25   

                       Open    Volume      macd  sma_10  sma_25        vwma  
Date                                                                         
2022-12-23 COM7.BK    31.75   4945800  0.150641   32.25   32.20   32.193966  
           EA.BK      95.00   8811400  0.208333   95.80   95.80   95.572557  
           KBANK.BK  144.00   7343700 -0.108974  143.55  143.38  143.548015  
           PTT.BK     31.50  37979900 -0.935897   31.50   32.48   31.501534  
           RCL.BK     30.50   2340300 -0.431090   29.65   29.87   29.645179